## 🔰PyTorchでニューラルネットワーク基礎 #22【文章分類・Transformer】

### 内容
* Qiitaの記事と連動しています
* Transformerを利用したテキスト分類
* 利用するデータ：data/data_90.zip
* data_90.zipは解凍して利用

### データについて
* webサイトへの「ログイン、登録、解約」の３種類の問い合わせを分類する60個データ
* 事前にmecabなどで形態素解析をして単語にIDを割り振り、等長の長さに変換してある
* <bos>文章<eos><pad>の形で長さが11になるまで<pad>を挿入
* カテゴリー数：３
* 単語数：146
* 系列長（文章の長さ）：11

**Transformer Encoderによるテキスト分類の基本構造**
* 埋め込み層→TransformerEncoder層→<BOS>へ集約→FC

**参考**
ネットワーク構造はBERT風にしてみました。
* Jacob Devlin, Ming-Wei Chang, Kenton Lee, Kristina Toutanova (2018) "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding"

**【検証精度】**
* 検証精度: 0.9
* D_MODEL = 16
* head = 4
* layer=6
* dim_feed=32


### 検証精度 たまたまほとんどが70〜80%　悪いと60%になるぞ
* 10回くらい試したけど、90%は1回だけ、ほとんど70〜80%の範囲。
* 数値を大きくするとほんのすこしだけ精度上昇っぽいが、1〜2個多く当たるだけ。テストデータ23個だしね。

### 傾向
* layer数を大きくすると精度が上昇しやすいけど、大きすぎてもNGっぽい
* 基本的に数値が大きいほど精度も緩やかな上昇傾向にあるけど、計算速度もどんどん遅くなるし、90%が多分上限ぽい
* 正規化やドロップアウト入れたほうが精度にばらつきが少ない傾向にある 
    ~~~python
    self.layer_norm = nn.LayerNorm(D_MODEL)
    self.dropout = nn.Dropout(0.1)
    ~~~

In [1]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [2]:
#デバイスの選択
device = "cuda" if torch.cuda.is_available() else "cpu"
print("利用デバイス:", device)

# 精度を計算する関数
def accuracy(y, t):
    _, argmax_list = torch.max(y, dim=1)
    accuracy = sum(argmax_list == t).item()/len(t)
    return accuracy

利用デバイス: cuda


In [ ]:
# データの読み込み データの保存場所に応じて変更してください
# x: IDベクトル
# t: ラベル
x = np.loadtxt("data_90/x_id_vector.csv", delimiter=",")
t = np.loadtxt("data_90/y_labels.csv", delimiter=",")
x = torch.LongTensor(x)
t = torch.LongTensor(t)

x, x_test, t, t_test = train_test_split(x,t, stratify=t,  random_state=55)

x = x.to(device)
t = t.to(device)
x_test = x_test.to(device)
t_test = t_test.to(device)

# 初期設定
WORDS = 146   # 単語数
SEQ_LEN = 11  # x.shape[1]  # 入力するIDベクトルの長さ
D_MODEL = 16  # 分散表現ベクトルの次元
CLASSES = 3   # 分類数

In [ ]:
x.shape, x_test.shape

(torch.Size([67, 11]), torch.Size([23, 11]))

In [19]:
import torch
import torch.nn as nn

class DNN(nn.Module):
    def __init__(self, pad_token_id: int=3): # ここでpadidng token idを指定。デフォルトは0
        super().__init__()
        self.pad_token_id = pad_token_id
       
        #  (1) トークン埋め込み <pad>を0に設定
        self.token_embedding = nn.Embedding(num_embeddings=WORDS, embedding_dim=D_MODEL,padding_idx=self.pad_token_id)
        
        # (2) 学習可能な位置埋め込み（0〜max_len-1）
        self.pos_embedding = nn.Embedding(num_embeddings=SEQ_LEN, embedding_dim=D_MODEL)
        
        # Layer Normalization と Dropout
        #self.layer_norm = nn.LayerNorm(D_MODEL)
        #self.dropout = nn.Dropout(0.1)
        
        # (3) Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=D_MODEL,
            nhead=4,
            dim_feedforward=32,
            dropout=0.1,
            batch_first=True,   # [batch, seq, d_model] で扱えるように
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer,num_layers=6)
        
        # (4) 文ベクトル → クラス数
        self.fc = nn.Linear(in_features=D_MODEL, out_features=3)
    
    def forward(self, x):
        # (5) TransformerEncoderの<pad>用マスク
        # pad ID=3でpadのところが True になるように mask を作成する
        src_key_padding_mask = (x == self.pad_token_id)
        
        # ---- 埋め込み ----
        # トークン埋め込み
        tok_emb = self.token_embedding(x)  # (batch, seq_len=11, d_model=16)    

        # 位置埋め込み 
        # ハードコードされている「11」は何を表しているの？
        # 文の長さが11。0〜10までの数値で単語の位置をあらわしています。
        pos_emb = self.pos_embedding(torch.arange(11, device=x.device))  # (seq_len=11, d_model=16)


        # (6) 分散表現行列＝トークン埋め込み + 位置埋め込み
        x = tok_emb + pos_emb.unsqueeze(0)     # (batch, seq_len=11, d_model=16)

               
        # (7) Transformer Encoder
        h = self.transformer_encoder(x, src_key_padding_mask=src_key_padding_mask)     
        
        # (8) <BOS>トークン（先頭）に情報を集約
        pooled = h[:, 0, :]  # [batch, d_model]  
        y = self.fc(pooled)  # [batch, num_labels=3]    
        return y

In [20]:
model = DNN()
model.to(device)

DNN(
  (token_embedding): Embedding(146, 16, padding_idx=3)
  (pos_embedding): Embedding(11, 16)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
        )
        (linear1): Linear(in_features=16, out_features=32, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=32, out_features=16, bias=True)
        (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc): Linear(in_features=16, out_features=3, bias=True)
)

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

In [22]:
LOOP = 50

for epoch in range(LOOP):
    optimizer.zero_grad()
    y = model(x)
    loss = criterion(y, t)
    acc  = accuracy(y,t)
    loss.backward()
    optimizer.step()

    if (epoch+1)%10==0:  
        print(f"{epoch}:\tloss:{loss.item():.3f}\tacc:{acc:.3f}")


9:	loss:1.074	acc:0.388
19:	loss:0.735	acc:0.806
29:	loss:0.305	acc:0.925
39:	loss:0.171	acc:0.955
49:	loss:0.067	acc:1.000


In [18]:
model.eval()
y_test = model(x_test)
acc = accuracy(y_test, t_test)
print(f"検証精度: {acc}")

検証精度: 0.9130434782608695


### 検証精度 たまたまほとんどが70〜80%　悪いと60%になるぞ
* 10回くらい試したけど、90%は1回だけ、ほとんど70〜80%の範囲。
* アップロードする前に試したときは75%くらいだった😅
* 数値を大きくするとほんのすこしだけ精度上昇っぽいが、1〜2個多く当たるだけ。テストデータ23個だしね。